# TestQuery Notebook

This is a test notebook to accompany the following post:

[Jupyter Notebooks with Elixir and RDF:<br/>Using IElixir in Jupyter Lab to explore SPARQL.ex]()
<br/>_– Tony Hammond ([@tonyhammond](https://twitter.com/tonyhammond))_

We shall be using [IElixir](https://github.com/pprzetacznik/IElixir) which implements a Jupyter kernel for Elixir. We'll also be using the Elixir package [SPARQL.ex](https://hex.pm/packages/sparql) which provides support for SPARQL querying over RDF datastores. And for further information on the [Jupyter Notebook](http://jupyter.org/) format and architecture see the project site.

![jupyter](../../priv/images/jupyter.png)

Here's a quick overview:

1. Setting up the environment
2. Simple queries
3. Install our TestQuery module
4. Testing it out
5. Etc.


## _A couple caveats first_

* _I am very new to this and not sure how best to configure `IElixir` for `Mix` projects. As a consequence I wasn't able to get the project tree and attributes loaded automatically but had to resort to loading the modules individually. I also wasn't able to use the `config.ex` file which would have allowed me to select for an alternative HTTP client, e.g. `hackney`, instead of the default `httpc` client. This meant that the default HTTP method and protocol needed to be overridden._

* _That aside, it all still seems to work. Although I am pretty sure it could be made to work even better._ 

## 1. Setting up the environment

&#x279C; ** Create `sparql_env`.**

IElixir uses the concept of virtual environments for managing packages. It uses [`Boyle`](https://github.com/pprzetacznik/IElixir#package-management-with-boyle) as its package manager.

Let's first create a `sparql_env` environment for our SPARQL dependencies using `Boyle.mk/1`. (Note also that there is a previously created an `rdf_env` environment set up for separately exploring [RDF.ex](https://hex.pm/packages/rdf) which we can just ignore.)

In [24]:
Boyle.mk("sparql_env")

{:ok, ["rdf_env", "sparql_env"]}

&#x279C; **Activate the environment.**

Next step is to activate the environment which will take care of compiling.

In [2]:
Boyle.activate("sparql_env")

All dependencies up to date


:ok

In [3]:
Boyle.list

{:ok, ["rdf_env", "sparql_env"]}

And we can double check that this is the current environment.

In [4]:
Boyle.active_env_name

"sparql_env"

&#x279C; **Install `sparql_client` dependencies.**

Next we install out dependencies.

In [5]:
Boyle.install({:sparql_client, "~> 0.2.1"})

Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  content_type 0.1.0
  decimal 1.5.0
  elixir_uuid 1.2.0
  jason 1.1.2
  json_ld 0.3.0
  mime 1.3.0
  nimble_csv 0.4.0
  rdf 0.5.1
  sparql 0.2.5
  sparql_client 0.2.1
  sweet_xml 0.6.5
  tesla 1.1.0
All dependencies up to date
==> rdf
Compiling 0 files (.erl)
Compiling 54 files (.ex)
Compiling vocabulary namespace for http://www.w3.org/2001/XMLSchema#
Compiling vocabulary namespace for http://www.w3.org/2001/XMLSchema#
Compiling vocabulary namespace for http://www.w3.org/1999/02/22-rdf-syntax-ns#
Compiling vocabulary namespace for http://www.w3.org/2000/01/rdf-schema#
Compiling vocabulary namespace for http://www.w3.org/2002/07/owl#
Compiling vocabulary namespace for http://www.w3.org/2004/02/skos/core#
==> sparql
Compiling 0 files (.erl)
Compiling 40 files (.ex)
Compiling vocabulary namespace for http://www.w3.org/2001/sw/DataAccess/tests/result-set#


:ok

&#x279C; **And check our packages.**

And lastly let's just sanity check that we do indeed have the `SPARQL` and `SPARQL.Client` modules installed.

In [6]:
exports SPARQL

execute_query/2                   execute_query/3                   query/1                           
query/2                           result_format/1                   result_format_by_extension/1      
result_format_by_media_type/1     result_formats/0                  


In [7]:
exports SPARQL.Client

__adapter__/0               __middleware__/0            default_accept_header/1     
delete/1                    delete/2                    delete/3                    
delete!/1                   delete!/2                   delete!/3                   
get/1                       get/2                       get/3                       
get!/1                      get!/2                      get!/3                      
head/1                      head/2                      head/3                      
head!/1                     head!/2                     head!/3                     
options/1                   options/2                   options/3                   
options!/1                  options!/2                  options!/3                  
patch/2                     patch/3                     patch/4                     
patch!/2                    patch!/3                    patch!/4                    
post/2                      post/3                      post/4   

Looks good.

## 2. Simple queries

&#x279C; **Let's try out a simple query.**


Let's choose a SPARQL endpoint. The ever-reliable [DBpedia](https://wiki.dbpedia.org/) will be good.

In [8]:
service = "http://dbpedia.org/sparql"

"http://dbpedia.org/sparql"

And we'll create a simple SPARQL query.

In [9]:
query = "select * where {?s ?p ?o} limit 5"

"select * where {?s ?p ?o} limit 5"

&#x279C; **First query try gives error.**

In [10]:
SPARQL.Client.query(query, service)

{:error, "unsupported result format for select query: \"text/html; charset=UTF-8\""}

&#x279C; **Let's check out the documentation.**

In [11]:
h SPARQL.Client.query


                   def query(query, endpoint, options \\ %{})                   

The query operation is used to send a SPARQL query to a service endpoint and
receive the results of the query.

The query can either be given as string or as an already parsed SPARQL.Query.

    with %SPARQL.Query{} = query <- SPARQL.Query.new("SELECT * WHERE { ?s ?p ?o }") do
      SPARQL.Client.query(query, "http://dbpedia.org/sparql")
    end

The type of the result returned depends on the query form:

  • SELECT queries will return a SPARQL.Query.ResultSet struct with a list
    of SPARQL.Query.Result structs in the results field.
  • ASK queries will return a SPARQL.Query.ResultSet struct with the
    boolean result in the results field
  • CONSTRUCT and DESCRIBE queries will return an RDF data structure

## Specifying the request method

The SPARQL 1.1 protocol spec defines three methods
(https://www.w3.org/TR/sparql11-protocol/#query-operation) to perform a SPARQL
query operation via HTTP, which c

We see that there are three supported methods:

* `GET`, `HTTP 1.1`
* `POST` (URL-encoded), `HTTP 1.0` – default
* `POST`, `HTTP 1.1`

Since the default URL-encoded `POST` method does not work, let's try the `GET` method.

&#x279C; **Second try succeeds – after setting :request_method (and :protocol_version).**

In [12]:
SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

{:ok, %SPARQL.Query.Result{results: [%{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax

Hey, that works! Let's run it again and capture the result this time.

In [13]:
{:ok, result} = SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

{:ok, %SPARQL.Query.Result{results: [%{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax

In [14]:
result |> SPARQL.Query.Result.get(:p) |> IO.inspect

[~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>]


[~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>]

Time for something more interesting. Let's try out our earlier `TestQuery` package.

## 3. Installing our TestQuery module

&#x279C; **Install TestQuery**

Is our `TestQuery` module loaded?

In [15]:
exports TestQuery

UndefinedFunctionError: 1

That'll be a no, then. So, let's explicitly import the module.

In [15]:
import_file("../../tonyhammond/examples/test_query/lib/test_query.ex")

Protocol.UndefinedError: 1

Rats!

Turns out that our call `:code.priv_dir(:test_query)}` is failing.

Let's just hardwire that for now and try again.

In [15]:
import_file("../../tonyhammond/examples/test_ipynb/lib/test_query.ex")

{:module, TestQuery, <<70, 79, 82, 49, 0, 0, 7, 104, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 191, 0, 0, 0, 18, 16, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 8, 95, 95, 105, 110, 102, ...>>, {:query, 2}}

And let's sanity check.

In [16]:
exports TestQuery

data/0      query/0     query/1     query/2     


Success!

&#x279C; **Install TestQuery.Client.** 

Now let's import the `TestQuery.Client` module.

In [17]:
import_file("../../tonyhammond/examples/test_query/lib/test_query/client.ex")

Protocol.UndefinedError: 1

Same problem. Same fix.

In [17]:
import_file("../../tonyhammond/examples/test_ipynb/lib/test_query/client.ex")

{:module, TestQuery.Client, <<70, 79, 82, 49, 0, 0, 26, 80, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 223, 0, 0, 0, 71, 23, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 46, 67, 108, 105, 101, 110, ...>>, {:_read_tuple, 1}}

And again sanity check.

In [18]:
exports TestQuery.Client

get_query/0          get_query_opts/0     get_service/0        hello/0              
read_table/1         rquery/0             rquery/1             rquery/2             
rquery_all/0         


## 4. Testing it out

&#x279C; **Now let's query**

In [19]:
import TestQuery

TestQuery

In [20]:
import TestQuery.Client

TestQuery.Client

In [21]:
hello

FunctionClauseError: 1

In [21]:
SPARQL.Client.query(get_query, get_service, get_query_opts)

{:ok, %SPARQL.Query.Result{results: [%{"o" => ~L"Hello World"en, "p" => ~I<http://www.w3.org/2000/01/rdf-schema#label>, "s" => ~I<http://dbpedia.org/resource/Hello_World>}], variables: ["s", "p", "o"]}}

In [22]:
exports SPARQL.Query.Result

__struct__/0          __struct__/1          add_identity/1        append/2              
get/2                 new/1                 new/2                 remove_identity/1     


In [23]:
SPARQL.Query.Result.get(result, :o) 

[~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>]

## 5. Etc.